This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'Tj9yhbRa8LR7wRLVYg5W'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [1]:
# First, import the relevant modules
import requests


In [15]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
pars = {'limit': 1}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=Tj9yhbRa8LR7wRLVYg5W', pars)
one_day_json = r.json()
print(one_day_json)
columns = one_day_json['dataset']['column_names']
one_day = one_day_json['dataset']['data']
print(columns)
print(one_day)

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-10-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-10-16T22:16:22.190Z', 'newest_available_date': '2020-10-16', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': 1, 'transform': None, 'column_index': None, 'start_date': '2000-06-07', 'end_date': '2020-10-16', 'data': [['2020-10-16', 118.0, 122.2, 117.8, 121.9, None, 186442.0, 22578015.5, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


The structure of the object (one_day_json) consists of an outer wrapping dict with one key ('dataset') and value (another nested dictionary). one_day_json['dataset'] is a dict where many keys point to metadata, the column names and data of interest are under the keys 'column_names' and 'data'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [38]:
# 1

pars = {'start_date': '2017-01-01', 'end_date': '2017-12-31'}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=Tj9yhbRa8LR7wRLVYg5W', pars)

# 2

data_2017 = r.json()['dataset']['data']

# 3, 4
lowest_opening = data_2017[0][1]
highest_opening = lowest_opening
largest_change_1d = data_2017[0][2] - data_2017[0][3]

for td in data_2017[1:]:  # td here means trading day
    if (td[1] != None):   # Skip None values
        lowest_opening = min(lowest_opening, td[1])
        highest_opening = max(highest_opening, td[1])
        largest_change_1d = max(largest_change_1d, td[2] - td[3])

# 5

largest_change_2d = data_2017[2][4] - data_2017[0][4]
for i in range(3, len(data_2017)):
    largest_change_2d = max(largest_change_2d, data_2017[i][4] - data_2017[i - 2][4])

# 6, 7

trading_volumes = []

for td in data_2017[1:]:  # td here means trading day
    if (td[6] != None):   # Skip None values
        trading_volumes.append(td[6])

n = len(trading_volumes)
trading_volume_avg = sum(trading_volumes) / n
trading_volumes.sort()
if (len(trading_volumes) % 2 == 0):
    trading_volumes_median = trading_volumes[n // 2 - 1] + trading_volumes[n // 2] / 2
else:
    trading_volumes_median = trading_volumes[n // 2]
    
print('The highest and lowest openings for AFX_X in 2017 were {} and {}, respectively'.format(lowest_opening, highest_opening))
print('The largest change during a day in this time period was {}'.format(largest_change_1d))
print('The largest change during a 2-day in this time period was {}'.format(largest_change_2d))
print('The average daily trading volume in this time period was {}'.format(trading_volume_avg))
print('The median daily trading volume in tis time period was {}'.format(trading_volumes_median))



The highest and lowest openings for AFX_X in 2017 were 34.0 and 53.11, respectively
The largest change during a day in this time period was 2.8100000000000023
The largest change during a 2-day in this time period was 3.1499999999999986
The average daily trading volume in this time period was 89338.84251968504
The median daily trading volume in tis time period was 114586.0
